<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/ScrapCharactercom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.z7g1slbkhw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.XjVmJcNrpH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.vZmtReiLTo/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as etc
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd
import multiprocessing as mp

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
url_links = 'https://www.character.com/pages/character-a-z'
content_main = requests.get(url_links).content

In [ ]:
#Initiating the BS4 parser
page = BeautifulSoup(content_main, 'html.parser')

In [ ]:
categorycharacter = page.find_all('a', class_="card__link")
len(categorycharacter)

In [ ]:
# category_links = []
# for data in categorycharacter:
#   base_link = "https://www.character.com/collections/"
#   data = data['href']
#   category = data.split('/')
#   category = category[-1]
#   full_link = base_link + category
#   category_links.append(full_link)

In [ ]:
# get all links in specific category
def get_links_category(url):
  all_links = []
  content = requests.get(url).content
  soup = BeautifulSoup(content, 'html.parser')
  product_card_div = soup.find_all('div', class_='stack stack--s align-center product-card__content')

  for link in product_card_div:
    # Find the link (a) element within the div
    link_element = link.find('a', class_='card__link')
    # Extract the href attribute from the link element
    base_link = "https://www.character.com"
    link = link_element['href']
    full_link = base_link + link
    all_links.append(full_link)
  return all_links




In [72]:
import json
def post_character(image, title_head,  final_description):
  try:
    description = title_head + " " + final_description
    url = 'https://jormungandr.wizewerx.tech/eat'
    headers = {
        'Authorization': 'p7RF3TwoFtaQ9ymoFMS3ZdYfSdc7hEoYedXxZ1L5l7vSXf59ntjvZTymoF5rfpy8RSpD4IErZt2ykSv64Xbu86l7',
        'Content-Type': 'application/json'
    }

    data = {
        "topic": "fuk",
        "description": description,
        "payload": {},
        "files": image
    }
    json_data = json.dumps(data)
    response = requests.post(url, headers=headers, data=json_data)
    if response.status_code == 200:
        pass
    else:
        pass
  except:
    pass

In [67]:
# # get detail individual page
def get_link_details(url):
  try:
    driver.get(url)
    try:
      all_images = driver.find_elements(By.CLASS_NAME, 'product__img.tns-item')
      final_image_links = []
      for element in all_images:
          image = element.find_element(By.TAG_NAME, "img")
          srcset = image.get_attribute("srcset")
          srcset_parts = srcset.split(',')
          first_url = srcset_parts[0].strip().split(' ')[0]
          image_link = first_url[2:]
          if image_link not in final_image_links:
            final_image_links.append(image_link)
    except:
      final_image_links = []
    try:
      header = driver.find_element(By.CSS_SELECTOR, 'header.flex.pad-items.justify-between.align-baseline')
      title = header.find_element(By.TAG_NAME, "h1")
      title_head = title.text
    except:
      title_head = ""
    try:
      description = driver.find_element(By.CSS_SELECTOR, 'div.rte.stack')
      description = description.text
      description = description.replace('\n', ' ')
      description = description.strip()
    except:
      description = ""
    post_character(final_image_links, title_head, description)
  except:
    pass

In [ ]:
# all_character_links = []
# for index, link  in enumerate(category_links):
#   print(index)
#   links_category = get_links_category(link)
#   all_character_links += links_category


In [ ]:
# len(all_character_links)

In [ ]:
# df = pd.DataFrame(all_character_links)
# df.head(3)

In [87]:
# # export to csv

# df.to_csv('character_com_all.csv', index=False)

In [68]:
df = pd.read_csv('/content/character_com_all.csv')

In [ ]:
df.describe()

In [70]:
df = df.drop_duplicates(keep='first')
df.reset_index(drop=True, inplace=True)

In [71]:
df.describe()

,0
count,3147
unique,3147
top,https://www.character.com/collections/disney/p...
freq,1


In [ ]:
# save and post data in db
for index, row in df.iterrows():
    get_link_details(row[0])
    print(index)